## Explore database and perform analysis from remote machine 
March 6, 2025

In [1]:
import sys
import os
import argparse
from sys import exit
import pprint
import pymongo
import xmltodict
from typing import Dict, Any
import itertools

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib widget
from ipywidgets import * 

In [3]:
## Import from package ( or import modules directly from repo ) 
from mgkdb import mgk_download
from mgkdb.support.mgk_login import mgk_login,f_login_dbase


## DB login

In [4]:
if __name__=="__main__":
    db_credentials='/Users/venkitesh_work/Documents/work/Sapient_AI/Data/mgkdb_data/db_credentials/mgktest1_remote_u1.pkl'
    login = f_login_dbase(db_credentials) 
    client, database = login.connect()
    ## Test extract 
    print("Collections",database.list_collection_names())

Collections ['LinearRuns', 'fs.files', 'NonlinRuns', 'ex.Nonlin', 'fs.chunks']


In [5]:
# Explore specific collection
# table='LinearRuns'
table='NonlinRuns'
assert table in database.list_collection_names(),f'{table} not in collections'
collection_name = database[table]

## Extract field entries

In [6]:
### IMAS plots 

# query_dict = {"gyrokineticsIMAS.species.1.density_norm": {"$gte": 1, "$lte": 2.1}}
query_dict = {'gyrokineticsIMAS.flux_surface.q':{"$gte": 2.0, "$lte": 2.1}}

output_field_strg='gyrokineticsIMAS'
selected_entries = [r[output_field_strg] for r in collection_name.find(query_dict,{output_field_strg:1, '_id':0})]

print("Number of collections", len(selected_entries))

Number of collections 13


## Get dataframe with imas fields

In [7]:

# Extract to dataframe 
df_output = pd.json_normalize(selected_entries)

# # Filter to get specific quantity
# column_strg = 'flux_surface'
# cols_select = [col for col in df_output.columns if col.startswith(column_strg)]
# df_output = df_output[cols_select]
# # Optional: Clean up column names by removing 'flux_surface.' prefix
# df_output.columns = [col.replace(f'{column_strg}.', '') for col in df_output.columns]

## convert columns with lists to numpy arrays for plots 
def is_col_list(series):
    return any(isinstance(x,list) for x in series.dropna())
list_cols = [col for col in df_output.columns if is_col_list(df_output[col])]

for col in list_cols: 
    df_output[col] = df_output[col].apply(np.array)


In [8]:
df_output

,max_repr_length,species,time,version.idspy_version,version.imas_dd_git_commit,version.imas_dd_version,version.idspy_internal_version,ids_properties.max_repr_length,ids_properties.version.idspy_version,ids_properties.version.imas_dd_git_commit,...,collisions.collisionality_norm,code.max_repr_length,code.version,code.name,code.description,code.commit,code.repository,code.parameters,code.output_flag,code.library
0,64,"[{'max_repr_length': 64, 'version': {'idspy_ve...","[0.4332496668013893, 0.8664993336027786, 1.299...",034000.2.0,845f1b30816f86a3cd4d53714dc56cdd307fdca1,03.40.00,2.0,64,034000.2.0,845f1b30816f86a3cd4d53714dc56cdd307fdca1,...,"[[0.0014510810177303828, 0.0014510810177303828...",64,None,CGYRO,,None,None,"<?xml version=""1.0"" encoding=""utf-8""?>\n<root>...",[],"[{'max_repr_length': 64, 'version': '0.7.1.dev..."
1,64,"[{'max_repr_length': 64, 'version': {'idspy_ve...","[0.4332496668013893, 0.8664993336027786, 1.299...",034000.2.0,845f1b30816f86a3cd4d53714dc56cdd307fdca1,03.40.00,2.0,64,034000.2.0,845f1b30816f86a3cd4d53714dc56cdd307fdca1,...,"[[0.0013834256868802877, 0.0013834256868802877...",64,None,CGYRO,,None,None,"<?xml version=""1.0"" encoding=""utf-8""?>\n<root>...",[],"[{'max_repr_length': 64, 'version': '0.7.1.dev..."
2,64,"[{'max_repr_length': 64, 'version': {'idspy_ve...","[0.4332496668013893, 0.8664993336027786, 1.299...",034000.2.0,845f1b30816f86a3cd4d53714dc56cdd307fdca1,03.40.00,2.0,64,034000.2.0,845f1b30816f86a3cd4d53714dc56cdd307fdca1,...,"[[0.0013291972721603258, 0.0013291972721603258...",64,None,CGYRO,,None,None,"<?xml version=""1.0"" encoding=""utf-8""?>\n<root>...",[],"[{'max_repr_length': 64, 'version': '0.7.1.dev..."
3,64,"[{'max_repr_length': 64, 'version': {'idspy_ve...","[0.4332496668013893, 0.8664993336027786, 1.299...",034000.2.0,845f1b30816f86a3cd4d53714dc56cdd307fdca1,03.40.00,2.0,64,034000.2.0,845f1b30816f86a3cd4d53714dc56cdd307fdca1,...,"[[0.0013291972721603258, 0.0013291972721603258...",64,None,CGYRO,,None,None,"<?xml version=""1.0"" encoding=""utf-8""?>\n<root>...",[],"[{'max_repr_length': 64, 'version': '0.7.1.dev..."
4,64,"[{'max_repr_length': 64, 'version': {'idspy_ve...","[0.4332496668013893, 0.8664993336027786, 1.299...",034000.2.0,845f1b30816f86a3cd4d53714dc56cdd307fdca1,03.40.00,2.0,64,034000.2.0,845f1b30816f86a3cd4d53714dc56cdd307fdca1,...,"[[0.0012544357960635924, 0.0012544357960635924...",64,None,CGYRO,,None,None,"<?xml version=""1.0"" encoding=""utf-8""?>\n<root>...",[],"[{'max_repr_length': 64, 'version': '0.7.1.dev..."
5,64,"[{'max_repr_length': 64, 'version': {'idspy_ve...","[0.4332496668013893, 0.8664993336027786, 1.299...",034000.2.0,845f1b30816f86a3cd4d53714dc56cdd307fdca1,03.40.00,2.0,64,034000.2.0,845f1b30816f86a3cd4d53714dc56cdd307fdca1,...,"[[0.0014587472269261261, 0.0014587472269261261...",64,None,CGYRO,,None,None,"<?xml version=""1.0"" encoding=""utf-8""?>\n<root>...",[],"[{'max_repr_length': 64, 'version': '0.7.1.dev..."
6,64,"[{'max_repr_length': 64, 'version': {'idspy_ve...","[0.4332496668013893, 0.8664993336027786, 1.299...",034000.2.0,845f1b30816f86a3cd4d53714dc56cdd307fdca1,03.40.00,2.0,64,034000.2.0,845f1b30816f86a3cd4d53714dc56cdd307fdca1,...,"[[0.001218313985684859, 0.001218313985684859, ...",64,None,CGYRO,,None,None,"<?xml version=""1.0"" encoding=""utf-8""?>\n<root>...",[],"[{'max_repr_length': 64, 'version': '0.7.1.dev..."
7,64,"[{'max_repr_length': 64, 'version': {'idspy_ve...","[0.4332496668013893, 0.8664993336027786, 1.299...",034000.2.0,845f1b30816f86a3cd4d53714dc56cdd307fdca1,03.40.00,2.0,64,034000.2.0,845f1b30816f86a3cd4d53714dc56cdd307fdca1,...,"[[0.0011632193292341305, 0.0011632193292341305...",64,None,CGYRO,,None,None,"<?xml version=""1.0"" encoding=""utf-8""?>\n<root>...",[],"[{'max_repr_length': 64, 'version': '0.7.1.dev..."
8,64,"[{'max_repr_length': 64, 'version': {'idspy_ve...","[0.4332496668013893, 0.8664993336027786, 1.299...",034000.2.0,845f1b30816f86a3cd4d53714dc56cdd307fdca1,03.40.00,2.0,64,034000.2.0,845f1b30816f86a3cd4d53714dc56c

## Get dataframe of input parameters

In [9]:

def parse_input_params(element: Dict[str, Any]) -> Dict[str, float]:
    """Parse XML string from element and convert to a dictionary of numeric values."""
    try:
        xml_strg = element['code']['parameters']
        data = xmltodict.parse(xml_strg)['root']
        return {k: float(v) if '.' in v else int(v) for k, v in data.items()}
    except (KeyError, ValueError) as e:
        print(f"Error processing element: {e}")
        return {}

# Create DataFrame directly from generator
df_inputs = pd.DataFrame.from_records(parse_input_params(l) for l in selected_entries)
# df_inputs

## Plots

In [18]:
def f_plot_fluxes(df,idxs,col):
    '''
    Plot dataframe columns 
    '''
    
    plt.figure()

    if df.loc[0,col].ndim==2:    
        for (count,idx),color in zip(enumerate(idxs),itertools.cycle('rbkcgmy')):
            for spec,marker in zip(range(df.loc[0,col].shape[0]),itertools.cycle('o*sxDhp')):
                if count==0:
                    plt.plot(df.loc[idx,col][spec],label=str(spec),linestyle='',marker=marker,color=color)
                else: 
                    plt.plot(df.loc[idx,col][spec],linestyle='',marker=marker,color=color)
                    
        plt.legend()
    else:
        for idx in idxs:
            print(idx)
            plt.plot(df.loc[idx,col],label='',linestyle='',marker='o')

    
    plt.title(col)
    plt.show()


selected_columns = [col for col in df_output.columns if ( col.startswith('non_linear') and isinstance(df_output.loc[0,col],np.ndarray) ) ] 

col='non_linear.fluxes_2d_k_x_sum.energy_a_field_parallel'
# col='non_linear.time_norm'
# f_plot_fluxes(df_output,[0,1],col)



In [19]:
selected_columns = [col for col in df_output.columns if ( col.startswith('non_linear') and isinstance(df_output.loc[0,col],np.ndarray) ) ] 

interact(f_plot_fluxes,df=fixed(df_output), \
         idxs = SelectMultiple(options=df_output.index.values, value=[0]),
         col = selected_columns
        )

interactive(children=(SelectMultiple(description='idxs', index=(0,), options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10…

<function __main__.f_plot_fluxes(df, idxs, col)>